In [1]:
import time
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import fashion_mnist
from sklearn.metrics import classification_report
import numpy as np
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
import PIL.Image as Image
import json
import pandas as pd
import logging

In [2]:
def data_loader(base_dir):
    images = []
    image_labels = []

    # Loop through each folder in the base directory (each folder corresponds to a label)
    for label in os.listdir(base_dir):
        label_dir = os.path.join(base_dir, label)

        # Check if the current item is a directory
        if os.path.isdir(label_dir):
            # Loop through all images in the folder
            for filename in os.listdir(label_dir):
                image_path = os.path.join(label_dir, filename)
                if filename.endswith('.png') or filename.endswith('.jpg'):
                    # Load the image and resize it to 28x28 (MNIST size)
                    img = Image.open(image_path).convert('L')
                    img_array = np.array(img)  # Convert image to numpy array
                    images.append(img_array)
                    image_labels.append(int(label))  # Use the folder name as the label

    # Convert lists to numpy arrays
    x_data = np.array(images)  # Shape: (num_images, 28, 28)
    y_data = np.array(image_labels)  # Shape: (num_images,)
    
    return x_data, y_data

In [3]:
# prepare data
dir_train_fmnist = r'C:\Users\81903\Desktop\coursework\thesis\6. repo\IMGRECO\week3-replicate\report\datasets\fmnist\equalized_train_images_labels-f1'
dir_test_fmnist = r'C:\Users\81903\Desktop\coursework\thesis\6. repo\IMGRECO\week3-replicate\report\datasets\fmnist\equalized_test_images_labels-f'
x_train, y_train = data_loader(dir_train_fmnist)
x_test, y_test = data_loader(dir_test_fmnist)
print(x_train.shape)
print(x_test.shape)

(60000, 28, 28)
(10000, 28, 28)


In [4]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)




In [5]:
# build the model
model = models.Sequential([
    layers.Conv2D(64, (2, 2), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    layers.Conv2D(64, (2, 2), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.25),

    layers.Dense(10, activation='softmax')  
])

# compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [6]:
# Create a log file for recording training results
log_filename = "training_baseline_cnn_fmnist.txt"
logging.basicConfig(filename=log_filename, level=logging.INFO, format="%(asctime)s - %(message)s")

# Log the start of the training process
logging.info("***************This is the logging book for FMNIST datasets******************")
# train the model

start_time = time.time()

history = model.fit(x_train, y_train, epochs=50, batch_size=128, validation_data=(x_test, y_test))

end_time = time.time()
training_time = end_time - start_time
print(f"training : {training_time:.2f} seconds")
logging.info(f"training : {training_time:.2f} seconds")

Epoch 1/50


469/469 [==============================] - 38s 76ms/step - loss: 0.6419 - accuracy: 0.7669 - val_loss: 0.3902 - val_accuracy: 0.8643
Epoch 2/50
469/469 [==============================] - 36s 78ms/step - loss: 0.4084 - accuracy: 0.8537 - val_loss: 0.3393 - val_accuracy: 0.8751
Epoch 3/50
469/469 [==============================] - 35s 75ms/step - loss: 0.3616 - accuracy: 0.8694 - val_loss: 0.3023 - val_accuracy: 0.8908
Epoch 4/50
469/469 [==============================] - 35s 75ms/step - loss: 0.3340 - accuracy: 0.8803 - val_loss: 0.2880 - val_accuracy: 0.8951
Epoch 5/50
469/469 [==============================] - 35s 75ms/step - loss: 0.3125 - accuracy: 0.8880 - val_loss: 0.2772 - val_accuracy: 0.8974
Epoch 6/50
469/469 [==============================] - 35s 74ms/step - loss: 0.3018 - accuracy: 0.8906 - val_loss: 0.2671 - val_accuracy: 0.9007
Epoch 7/50
469/469 [==============================] - 35s 74ms/step - loss: 0.2877 - accuracy: 0.8954 - val_loss: 0.2592 - val_accurac

In [7]:
# Evaluate the model and measure inference time
start_time = time.time()  # Start time

y_pred = model.predict(x_test)  # Model predictions
y_pred_classes = np.argmax(y_pred, axis=1)  # Get the class with the highest probability

end_time = time.time()  # End time
inference_time = end_time - start_time  # Calculate the time taken for inference

# Print the inference time
print(f"Inference time: {inference_time:.4f} seconds")
logging.info(f"Inference time: {inference_time:.4f} seconds")

# Precision、Recall、F1-score
report = classification_report(y_test, y_pred_classes, target_names=[str(i) for i in range(10)])
print("\nclassification report:\n", report)
logging.info("\nclassification report:\n", report)
# Accuracy 
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f'\nAccuracy: {test_acc:.4f}')
logging.info(f'\nAccuracy: {test_acc:.4f}')

313/313 [==============================] - 3s 9ms/step
Inference time: 5.4689 seconds

classification report:
               precision    recall  f1-score   support

           0       0.86      0.88      0.87      1000
           1       0.99      0.98      0.99      1000
           2       0.88      0.88      0.88      1000
           3       0.91      0.92      0.92      1000
           4       0.85      0.91      0.88      1000
           5       0.99      0.98      0.99      1000
           6       0.79      0.72      0.75      1000
           7       0.95      0.98      0.97      1000
           8       0.97      0.98      0.98      1000
           9       0.98      0.96      0.97      1000

    accuracy                           0.92     10000
   macro avg       0.92      0.92      0.92     10000
weighted avg       0.92      0.92      0.92     10000



--- Logging error ---
Traceback (most recent call last):
  File "D:\software\anaconda\Lib\logging\__init__.py", line 1110, in emit
    msg = self.format(record)
          ^^^^^^^^^^^^^^^^^^^
  File "D:\software\anaconda\Lib\logging\__init__.py", line 953, in format
    return fmt.format(record)
           ^^^^^^^^^^^^^^^^^^
  File "D:\software\anaconda\Lib\logging\__init__.py", line 687, in format
    record.message = record.getMessage()
                     ^^^^^^^^^^^^^^^^^^^
  File "D:\software\anaconda\Lib\logging\__init__.py", line 377, in getMessage
    msg = msg % self.args
          ~~~~^~~~~~~~~~~
TypeError: not all arguments converted during string formatting
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "D:\software\anaconda\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "D:\software\anaconda\Lib\site-packages\traitlets\config\application.py", li

313/313 - 3s - loss: 0.2496 - accuracy: 0.9183 - 3s/epoch - 10ms/step

Accuracy: 0.9183


In [9]:


model.save('cnn_model_fmnist.h5')

history_dict = history.history
with open("training_history_fmnist.json", "w") as f:
    json.dump(history_dict, f)

# Convert the classification report string to a dictionary
report_dict = classification_report(y_test, y_pred_classes, target_names=[str(i) for i in range(10)], output_dict=True)
report_df = pd.DataFrame(report_dict).transpose()
report_df.to_csv("classification_report_fmnist.csv", index=True)

with open("model_info_fmnist.txt", "w") as f:
    f.write(f"training time: {training_time:.2f} s\n")
    f.write(f"accuracy on test: {test_acc:.4f}\n")

D:\software\anaconda\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
